In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2 as pg
from io import StringIO
from datetime import datetime
from enum import Enum
import time
import requests
%matplotlib inline

In [2]:
connection = pg.connect(user="admin",
                            password="quest",
                            host="127.0.0.1",
                            port="8812",
                            database="qdb",
                            options='-c statement_timeout=300000')

In [3]:
SENSOR_TO_TABLE_NAME = {

    'accelerometeruncalibrated': 'acc',
    'gyroscopeuncalibrated': 'gyro',
    'magnetometeruncalibrated': 'mag'

}

In [4]:
def write_sensor_payloads(data:dict, table_name:str):
    
    """
    Write phone sensor data to database tables

    Parameters
    ----------
    data : dict
        The raw request data sent by the phone
    table_name : str
        The name of the table to write to 
    """

    session_id = data['sessionId']
    device_id = data['deviceId']

    # Create an empty dict to store structured sensor from the payload
    structured_payload = {'device_id':[],
                            'session_id':[],
                            'device_timestamp':[],
                            'recorded_timestamp':[],
                            'sensor_name':[],
                            'x':[],
                            'y':[],
                            'z':[]
                            }
    
    for d in data['payload']:

        # Triaxial sensors
        if d.get("name") in ['accelerometeruncalibrated','gyroscopeuncalibrated','magnetometeruncalibrated']:

            structured_payload['device_id'].append(device_id)
            structured_payload['session_id'].append(session_id)
            structured_payload['device_timestamp'].append(str(datetime.fromtimestamp(int(d["time"]) / 1000000000)))
            structured_payload['recorded_timestamp'].append(str(datetime.utcnow()))
            structured_payload['sensor_name'].append(SENSOR_TO_TABLE_NAME.get(d.get("name")))
            structured_payload['x'].append(d["values"]["x"])
            structured_payload['y'].append(d["values"]["y"])
            structured_payload['z'].append(d["values"]["z"])  

    output = StringIO()
    pd.DataFrame(structured_payload).to_csv(output, sep=',', header=True, index=False)
    output.seek(0)
    contents = output.getvalue()
    csv = {'data': ('device_offload', contents)}
    server = 'http://localhost:9000/imp'
    response = requests.post(server, files=csv)


In [5]:
data = pd.read_sql("select * from device_offload", connection)
data.sort_values('recorded_timestamp')

/var/folders/9r/y3yrm7yx2x31dxvgqt8pww3h0000gn/T/ipykernel_43662/2068274001.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql("select * from device_offload", connection)


,device_id,session_id,device_timestamp,recorded_timestamp,sensor_name,x,y,z
0,asdfsadf,asdfsfff,1969-12-31 18:00:01.669583,2022-11-28 15:15:56.776696,acc,43.000,31.000,12.000
1,asdfsadf,asdfsfff,1969-12-31 18:00:01.669583,2022-11-28 15:15:56.776707,gyro,43.000,31.000,12.000
2,asdfsadf,asdfsfff,1969-12-31 18:00:01.669583,2022-11-28 15:15:56.776713,mag,43.000,31.000,12.000
3,asdfsadf,asdfsfff,1969-12-31 18:00:01.669583,2022-11-28 15:16:49.918553,acc,43.000,31.000,12.000
4,asdfsadf,asdfsfff,1969-12-31 18:00:01.669583,2022-11-28 15:16:49.918563,gyro,43.000,31.000,12.000
...,...,...,...,...,...,...,...,...
531,86a5b0e3-6e06-40e2-b226-5a72bd39b65b,4225c45f-e701-4d03-af7f-ae3932751fd5,2022-11-30 16:01:10.776309,2022-11-30 16:01:10.870112,acc,0.105,-0.473,-0.872
532,86a5b0e3-6e06-40e2-b226-5a72bd39b65b,4225c45f-e701-4d03-af7f-ae3932751fd5,2022-11-30 16:01:10.796365,2022-11-30 16:01:10.870121,acc,0.098,-0.469,-0.867
533,86a5b0e3-6e06-40e2-b226-5a72bd39b65b,4225c45f-e701-4d03-af7f-ae3932751fd5,2022-11-30 16:01:10.816422,2022-11-30 16:01:10.870128,acc,0.108,-0.470,-0.877
534,86a5b0e3-6e06-40e2-b226-5a72bd39b65b,4225c45f-e701-4d03-af7f-ae3932751fd5,2022-11-30 16:01:10.836478,2022-11-30 16:01:10.870134,acc,0.102,-0.470,-0.884
